<a href="https://colab.research.google.com/github/elevate-foundry/braille-multimodal-substrate/blob/main/colab_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔤 Braille-Native Cognition: LoRA Fine-Tuning

Train a model to **think in 8-dot braille** - understanding text, images, audio, video, and binary files as braille patterns.

**Novel contribution:** First LLM trained to recognize binary file signatures (ZIP, PNG, etc.) as braille.

## Setup
1. Runtime → Change runtime type → **T4 GPU** (free) or **A100** (Colab Pro)
2. Run all cells

In [ ]:
# Install dependencies
!pip install -q unsloth transformers datasets peft accelerate bitsandbytes trl
!pip install -q huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 742.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 142.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Check GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Training Data

133 examples covering:
- Text ↔ Braille encoding/decoding
- Binary file signatures (ZIP, PNG, JPEG, PDF, etc.)
- Multimodal understanding (images, audio as braille)
- Tutoring scenarios

In [ ]:
# Download training data from GitHub
!wget -q https://raw.githubusercontent.com/elevate-foundry/braille-multimodal-substrate/main/lora_training/braille_alpaca.json

import json
with open('braille_alpaca.json', 'r') as f:
    training_data = json.load(f)

print(f"Loaded {len(training_data)} training examples")
print(f"\nExample:")
print(json.dumps(training_data[0], indent=2))

In [ ]:
# Format for training
def format_prompt(example):
    instruction = example["instruction"]
    input_text = example.get("input", "")
    output = example["output"]

    if input_text:
        return f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        return f"""### Instruction:
{instruction}

### Response:
{output}"""

# Create dataset
from datasets import Dataset
formatted_data = [{"text": format_prompt(ex)} for ex in training_data]
dataset = Dataset.from_list(formatted_data)
print(f"Dataset ready: {len(dataset)} examples")

## Load Model with Unsloth (2x faster training)

In [ ]:
from unsloth import FastLanguageModel

# Choose model size based on GPU
# T4 (16GB): Use 3B or smaller
# A100 (40GB): Can use 7B+

max_seq_length = 2048
model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"  # 3B params, 4-bit quantized

print(f"Loading {model_name}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

print("Model loaded!")

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

model.print_trainable_parameters()

## Train!

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./braille_lora_output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=5,  # More epochs for better learning
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
    save_strategy="epoch",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
)

print("Starting training...")
trainer.train()
print("Training complete!")

## Test the Model

In [ ]:
# Test prompts
test_prompts = [
    "What is the braille pattern for the letter 'h'?",
    "What file type does this braille header indicate: ⡐⡋⠃⠄",
    "Decode this braille: ⠓⠑⠇⠇⠕",
    "How can braille represent any binary file?",
    "Explain the structure of a ZIP file in braille encoding",
]

FastLanguageModel.for_inference(model)

for prompt in test_prompts:
    print("=" * 50)
    print(f"Q: {prompt}")

    formatted = f"### Instruction:\n{prompt}\n\n### Response:"
    inputs = tokenizer(formatted, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("### Response:")[-1].strip()
    print(f"A: {response[:300]}...")
    print()

## Save & Export

In [ ]:
# Save LoRA adapters
model.save_pretrained("braille_lora_adapters")
tokenizer.save_pretrained("braille_lora_adapters")
print("LoRA adapters saved!")

In [ ]:
# Save merged model (for Ollama)
model.save_pretrained_merged("braille_merged", tokenizer, save_method="merged_16bit")
print("Merged model saved!")

In [ ]:
# Convert to GGUF for Ollama
# This creates a file you can use with: ollama create braille-native -f Modelfile

model.save_pretrained_gguf("braille_gguf", tokenizer, quantization_method="q4_k_m")
print("GGUF model saved! Download braille_gguf/ and use with Ollama.")

In [ ]:
# Optional: Push to Hugging Face Hub
# from huggingface_hub import login
# login(token="your_token")
# model.push_to_hub("your-username/braille-native-3b-lora")

## Download Files

After training, download:
1. `braille_gguf/` - For Ollama
2. `braille_lora_adapters/` - LoRA weights only

To use with Ollama:
```bash
# Create Modelfile
echo 'FROM ./braille_gguf/model.gguf' > Modelfile
echo 'SYSTEM "You are a Braille-Native AI..."' >> Modelfile

# Create model
ollama create braille-native -f Modelfile

# Test
ollama run braille-native "Decode: ⠓⠑⠇⠇⠕"
```

In [ ]:
# Zip for download
!zip -r braille_gguf.zip braille_gguf/
!zip -r braille_lora_adapters.zip braille_lora_adapters/

from google.colab import files
# files.download('braille_gguf.zip')  # Uncomment to download